In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import sys
import gc
import os
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, MOCK_LLAMA_CONFIG, WrapperConfig

In [ ]:
from dataclasses import replace
cfg = replace(MOCK_LLAMA_CONFIG, max_samples=1000, single_process_mode=True)
wrapper = TrainerWrapper(MOCK_LLAMA_CONFIG)
wrapper.init_model()

In [ ]:
wrapper.init_data_module()

In [25]:
wrapper.init_trainer()

2024-12-17 03:57:04.527 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:139 - LoRA already loaded
2024-12-17 03:57:04.674 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:184 - Initializing DPOTrainer, with project name: codecontests-llama-3b, run_name: run-382863
2024-12-17 03:57:05.211 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:200 - Loading cached logprobs...


In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [29]:
wrapper.trainer.max_new_tokens = 138
wrapper.train()

2024-12-17 04:00:08.456 | INFO     | trl_wrapper.trainer_wrapper:train:288 - Starting training.
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x14c906c77600>
Traceback (most recent call last):
  File "/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1568, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.11/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 948, in wait
    ready = selector.select(timeout)
        

KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
samples_sorted_logprob_diff = out_df.sort_values(
    "logprob_differences", ascending=False
)
samples_sorted_highest_diff = samples_sorted_logprob_diff[["prompt", "chosen", "rejected", "logprob_differences"]].head(10)
for prompt, chosen, rejected, logprob_diff in samples_sorted_highest_diff.values:
    display(Markdown(f"\n### Prompt: {prompt}\n\n### Chosen:\n {chosen}\n\n### Rejected:\n {rejected}\n\nLogprob diff: {logprob_diff}"))